In [1]:
# @Sarthak_Manchanda
# Implementation of Mask-RCNN in Python-2.x
import datetime 
import math 
import os 
import random 
import re

import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import torch.utils.data
from torch.autograd import Variable 

# import utils
# import visualize 
# from nms.nms_wrapper import nms 
# from roialign.roi_align.crop_and_resize import CropAndResizeFunction

In [9]:
# Logging Utility Function
###########################
def log(text, array=None):
    # prints text message.
    # Also prints numpy array shape, min, max values.
    if array is not None:
        text = text.ljust(25)
        text += ("shape: {:20} min:{:10.5f} max:{:10.5f}".format(
            str(array.shape),
            array.min() if array.size else "",
            array.max() if array.size else "" ) )
    print(text)
    
    
def printProgressBar(iteration, total, prefix='', suffix='', decimals=1, length=100, fill='*'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration/float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length-filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix))
    if iteration == total:
        print()

In [11]:
# Pytorch Utility Functions 
############################

# Function 1
def unique1d(tensor):
    if tensor.size()[0] == 0 or tensor.size()[0] == 1:
        return tensor
    
    tensor = tensor.sort()[0]
    unique_bool = tensor[1:] != tensor[:-1]
    first_element = Variable(torch.ByteTensor([True]), requires_grad=True)
    
    if tensor.is_cuda:
        first_element = first_element.cuda()
    
    unique_bool = torch.cat((first_element, unique_bool), dim=True)
    return tensor[unique_bool.data]

# Function 2
def intersect1d(tensor1, tensor2):
    aux = torch.cat((tensor1, tensor2), dim=True)
    aux = aux.sort()[0]
    return aux[:-1][(aux[1:]==aux[:-1]).data]

# Function 3
def log2(x):
    """Log to the base 2 is not natively implemented in PyTorch"""
    ln2 = Variable(torch.log(torch.FloatTensor([2.0])), requires_grad=False)
    if x.is_cuda:
        ln2 = ln2.cuda()
    return torch.log(x)/log2

# Class 1
class samePad2d(nn.Module):
    """For padding similar to in TensorFlow"""
    def __init__(self, kernel_size, stride):
        super(SamePad2d, self).__init__()
        self.kernel_size = torch.nn.modules.utils._pair(kernel_size) #??
        self.stride = torch.nn.modules.utils._pair(stride)
        
    def forward(self, input):
        in_width = input.size()[2]
        in_height = input.size()[3]
        out_width = math.ceil(float(in_width) / float(self.stride[0]))
        out_height = math.ceil(float(in_height) / float(self.stride[1]))
        pad_along_width = ((out_width - 1)*self.stride[0] + self.kernel_size[0]-in_width) #??
        pad_along_height = ((out_height - 1)*self.stride[1] + self.kernel_size[1]-in_height)
        pad_left = math.floor(pad_along_width / 2)
        pad_top = math.floor(pad_along_height / 2)
        pad_right = pad_along_width - pad_left
        pad_bottom = pad_along_height - pad_top
        return F.pad(input,(pad_left,pad_right,pad_top,pad_bottom),'constant',0)
    
    def __repr__(self):
        return self.__class__.__name__

In [ ]:
# Feature Pyramid Network(FPN) Graph
#####################################
